In [3]:
import pyproj
import geopandas as gpd
import numpy as np
import rasterio
from rasterio import features
from shapely.geometry import box
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score

In [ ]:

pyproj.datadir.set_data_dir("/home/yifan/anaconda3/envs/myenv/share/proj")
print(pyproj.datadir.get_data_dir())

In [9]:
############################
# 1. 栅格化函数
############################
def rasterize_polygon(gdf, transform, width, height, fill_value=0, dtype='int16'):
    """
    将GeoDataFrame中的几何体栅格化为给定范围和分辨率的numpy数组。
    
    参数:
    - gdf: 要栅格化的GeoDataFrame
    - transform: rasterio.transform 对象
    - width, height: 输出栅格的宽和高
    - fill_value: 未覆盖区域的像素值
    - dtype: 输出栅格的数据类型
    
    返回:
    - raster: 栅格化后的numpy数组
    """
    shapes = [(geom, 1) for geom in gdf.geometry]
    raster = features.rasterize(
        shapes,
        out_shape=(height, width),
        transform=transform,
        fill=fill_value,
        dtype=dtype
    )
    return raster

In [ ]:
############################
# 2. 读取并获取检测结果范围
############################
def get_detection_raster_params(detection_shp_path, pixel_size=0.0001, raster_width=256, raster_height=256):
    """
    根据检测结果的矢量，获取栅格化所需的 transform、width、height 等参数。
    
    参数:
    - detection_shp_path: 检测结果的矢量文件路径
    - pixel_size: 像素分辨率（可选）
    - raster_width: 输出栅格的宽度（默认256）
    - raster_height: 输出栅格的高度（默认256）
    
    返回:
    - transform: rasterio.transform 对象
    - width: 栅格宽度
    - height: 栅格高度
    """
    detection_gdf = gpd.read_file(detection_shp_path)
    
    # 获取检测结果的边界框
    bounds = detection_gdf.total_bounds  # (minx, miny, maxx, maxy)
    minx, miny, maxx, maxy = bounds
    
    # 定义栅格的仿射变换
    transform = rasterio.transform.from_bounds(minx, miny, maxx, maxy, raster_width, raster_height)
    
    return transform, raster_width, raster_height

In [ ]:
############################
# 3. 删除非森林区域像素
############################
def mask_non_forest_pixels(prediction_raster, forest_raster):
    """
    将不在森林范围（forest_raster=0）内的像素置为0。
    
    参数:
    - prediction_raster: 预测结果栅格（numpy数组）
    - forest_raster: 森林掩膜栅格（numpy数组）
    
    返回:
    - masked_result: 处理后的预测栅格
    """
    masked_result = prediction_raster.copy()
    masked_result[forest_raster == 0] = 0
    return masked_result

In [ ]:
############################
# 4. 计算评估指标（手动方法）
############################
def calculate_metrics_between_rasters(pred_raster, true_raster):
    """
    计算两个二值栅格（预测与真实标注）之间的 TP, FP, FN, Precision, Recall, F1-score, IoU。
    
    参数:
    - pred_raster: 预测结果栅格（numpy数组，1表示正类，0表示负类）
    - true_raster: 真实标注栅格（numpy数组，1表示正类，0表示负类）
    
    返回:
    - TP: 真正例的数量
    - FP: 假正例的数量
    - FN: 假负例的数量
    - precision: 精确率
    - recall: 召回率
    - f1_score: F1 分数
    - iou: 交并比 (Intersection over Union)
    """
    assert pred_raster.shape == true_raster.shape, "两个栅格的尺寸必须相同"

    # 计算 TP, FP, FN
    TP = np.sum((pred_raster == 1) & (true_raster == 1))
    FP = np.sum((pred_raster == 1) & (true_raster == 0))
    FN = np.sum((pred_raster == 0) & (true_raster == 1))

    # 计算 Precision, Recall, F1-score, IoU
    if TP + FP > 0:
        precision = TP / (TP + FP)
    else:
        precision = 0.0

    if TP + FN > 0:
        recall = TP / (TP + FN)
    else:
        recall = 0.0

    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0.0

    if TP + FP + FN > 0:
        iou = TP / (TP + FP + FN)
    else:
        iou = 0.0

    return TP, FP, FN, precision, recall, f1, iou

In [ ]:
############################
# 5. 计算评估指标（使用 sklearn）
############################
def calculate_metrics_with_sklearn(pred_raster, true_raster):
    """
    使用 sklearn 库计算两个二值栅格（预测与真实标注）之间的 Precision, Recall, F1-score 和 IoU。
    
    参数:
    - pred_raster: 预测结果栅格（numpy数组，1表示正类，0表示负类）
    - true_raster: 真实标注栅格（numpy数组，1表示正类，0表示负类）
    
    返回:
    - precision: 精确率
    - recall: 召回率
    - f1: F1 分数
    - iou: 交并比 (Intersection over Union)
    """
    # 展平栅格以进行像素级别的对比
    pred_flat = pred_raster.flatten()
    true_flat = true_raster.flatten()

    # 只考虑 difference_raster 和 merged_raster 中非0的部分，避免计算过程中包含无效区域
    valid_mask = (pred_flat != 0) | (true_flat != 0)

    y_true = true_flat[valid_mask]
    y_pred = pred_flat[valid_mask]

    # 二值化预测结果（确保为0或1）
    y_pred = (y_pred > 0).astype('int16')
    y_true = (y_true > 0).astype('int16')

    # 检查 y_true 和 y_pred 的分布，避免全为一类的情况
    if y_true.sum() == 0 or y_pred.sum() == 0:
        print("警告：y_true 或 y_pred 中没有正样本或负样本，可能导致召回率或精确率异常。")
        return None, None, None, None

    # 计算 Precision, Recall, F1-score
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    # 计算 IoU
    iou = jaccard_score(y_true, y_pred, zero_division=0)

    return precision, recall, f1, iou

In [ ]:
############################
# 6. 主流程示例
############################
def main_evaluation(forest_mask_path, annotation_path, detection_result_path, pixel_size=0.0001):
    """
    主流程:
    1) 读取和栅格化 detection_result, forest_mask, annotation 到相同范围与大小
    2) 将 detection_result 中不属于森林区域的像素置为0
    3) 与 annotation 比较并计算评估指标（手动和 sklearn 方法）
    
    参数:
    - forest_mask_path: 森林掩膜的矢量文件路径
    - annotation_path: 手动标注的矢量文件路径
    - detection_result_path: 检测结果的矢量文件路径
    - pixel_size: 像素分辨率（可选）
    """
    
    # 获取栅格范围和大小
    transform, width, height = get_detection_raster_params(
        detection_result_path,
        pixel_size=pixel_size,
        raster_width=256,
        raster_height=256
    )
    
    # 读取矢量文件
    forest_gdf = gpd.read_file(forest_mask_path)
    annotation_gdf = gpd.read_file(annotation_path)
    detection_gdf = gpd.read_file(detection_result_path)
    
    # 栅格化：森林掩膜
    forest_raster = rasterize_polygon(forest_gdf, transform, width, height)
    forest_raster = (forest_raster > 0).astype('int16')  # 二值化
    
    # 栅格化：标注
    annotation_raster = rasterize_polygon(annotation_gdf, transform, width, height)
    annotation_raster = (annotation_raster > 0).astype('int16')  # 二值化
    
    # 栅格化：检测结果
    detection_raster = rasterize_polygon(detection_gdf, transform, width, height)
    detection_raster = (detection_raster > 0).astype('int16')  # 二值化
    
    # 删除不属于森林的像素
    masked_detection_raster = mask_non_forest_pixels(detection_raster, forest_raster)
    
    # 计算评估指标（手动方法）
    TP, FP, FN, precision_manual, recall_manual, f1_manual, iou_manual = calculate_metrics_between_rasters(
        masked_detection_raster,
        annotation_raster
    )
    
    print("评估结果（手动计算）:")
    print(f"TP: {TP}")
    print(f"FP: {FP}")
    print(f"FN: {FN}")
    print(f"Precision: {precision_manual:.4f}")
    print(f"Recall: {recall_manual:.4f}")
    print(f"F1-Score: {f1_manual:.4f}")
    print(f"IoU: {iou_manual:.4f}")
    
    # 计算评估指标（使用 sklearn）
    precision_sklearn, recall_sklearn, f1_sklearn, iou_sklearn = calculate_metrics_with_sklearn(
        masked_detection_raster,
        annotation_raster
    )
    
    if precision_sklearn is not None:
        print("\n评估结果（使用 sklearn）:")
        print(f"Precision: {precision_sklearn:.4f}")
        print(f"Recall: {recall_sklearn:.4f}")
        print(f"F1-Score: {f1_sklearn:.4f}")
        print(f"IoU: {iou_sklearn:.4f}")

In [ ]:
if __name__ == "__main__":
    
    forest_mask = "forest_mask.shp"
    annotation = "annotation.shp"
    detection_result = "detection_result.shp"
    
    main_evaluation(forest_mask, annotation, detection_result)